In [4]:
from open3d import *
import numpy as np
import os
import sys
import random
import math
import glob
import matplotlib.pyplot as plt
import scipy.interpolate as si
from ipywidgets import FloatProgress
from IPython.display import display
import splipy as sp
import bisect
from collections import OrderedDict
import matplotlib.tri as tri
from shapely.geometry import LineString
from shapely.geometry import Polygon
from shapely.ops import linemerge

In [5]:
def read_mesh(meshdir):
    if os.path.isfile(meshdir):
        print('Mesh File successfully loaded')
        mesh = read_triangle_mesh(meshdir)
        print('Vertices: '+str(len(np.asarray(mesh.vertices))))
        print('Triangles: '+str(len(np.asarray(mesh.triangles))))
        return mesh
    else:
        print('File Directory does not exist')

def read_pcd(filedir):
    if os.path.isfile(filedir):
        print('Point Cloud File successfully loaded')
        return read_point_cloud(filedir)
    else:
        print('File Directory does not exist')

def write_pcd(pcd, filedir):
    if not os.path.isfile(filedir):
        print('Point Cloud File successfully created')
        write_point_cloud(filedir, pcd)
    else:
        print('File Directory already exists')
        
def create_mesh(vertices, triangles):
    mesh = TriangleMesh()
    mesh.vertices = Vector3dVector(vertices)
    mesh.triangles = Vector3iVector(triangles)
    mesh.compute_triangle_normals()
    mesh.compute_vertex_normals()
    return mesh

def create_pcd(pts):
    pcd = PointCloud()
    pcd.points = Vector3dVector(pts)
    return pcd

def rotate_pcd(pcd, theta):
    points = np.asarray(pcd.points)
    r_x = np.array([[1,0,0],[0,math.cos(theta[0]),-math.sin(theta[0])],[0,math.sin(theta[0]),math.cos(theta[0])]])
    r_y = np.array([[math.cos(theta[1]),0,math.sin(theta[1])],[0,1,0],[-math.sin(theta[1]),0,math.cos(theta[1])]])
    r_z = np.array([[math.cos(theta[2]),-math.sin(theta[2]),0],[math.sin(theta[2]),math.cos(theta[2]),0],[0,0,1]])   
    r = np.dot(r_z, np.dot(r_y,r_x))
    points = np.matmul(points, r)
    pcd.points = Vector3dVector(points)
    return pcd

def pcd_processing(pcd, angle):
    angle = math.radians(angle)
    points = np.asarray(pcd.points)
    x = points[:,0]
    y = points[:,2] + abs(points[:,2].min())
    z = points[:,1]
    pcd = create_pcd(np.array([x, y, z]).transpose())
    return rotate_pcd(pcd, [0,angle,0])

In [19]:
def closest_distance(points, max_neighbors):
    distances = []
    tree = KDTreeFlann(create_pcd(points))
    for point in points:
        [_, _, n_distances] = tree.search_knn_vector_3d(point, max_neighbors+1)
        for d in n_distances[1:]:
            distances.append(np.sqrt(d))
    return max(distances)

def slice_cutoffs(slices, max_neighbors):
    print('Finding Slice Cutoffs')
    f = FloatProgress(min=0, max=len(slices)-1)
    display(f)
    cutoffs = []
    for i in range(len(slices)):
        negx_cutoff = None
        posx_cutoff = None
        current_slice = slices[i]
        assert (len(current_slice) > 0), f'Slice {i} is empty'
        tolerance = 0.5*closest_distance(current_slice, max_neighbors)
        assert (tolerance > 0), f'Slice {i} has no tolerance'
        increment = tolerance/4
        min_x = current_slice[:,0].min()
        max_x = current_slice[:,0].max()
        mean_x = current_slice[:,0].mean()
        current_x = min_x
        while current_x < mean_x + tolerance:
            current_interval = current_slice[current_slice[:,0] >= current_x - tolerance]
            current_interval = current_interval[current_interval[:,0] <= current_x + tolerance]
            if len(current_interval) == 0:
                negx_cutoff = current_x
                current_x = mean_x + tolerance
                break
            current_x += increment
        current_x = max_x
        while current_x > mean_x - tolerance:
            current_interval = current_slice[current_slice[:,0] >= current_x - tolerance]
            current_interval = current_interval[current_interval[:,0] <= current_x + tolerance]
            if len(current_interval) == 0:
                posx_cutoff = current_x
                current_x = mean_x - tolerance
                break
            current_x -= increment
        cutoffs.append([negx_cutoff,posx_cutoff])
        f.value = i
    return cutoffs

def cut_slices(slices, cutoffs, lower, upper_negx, upper_posx):
    print('Cutting Slices')
    f = FloatProgress(min=lower+1, max=len(slices)-1)
    display(f)
    new_slices = []
    if upper_negx > upper_posx:
        upper = upper_negx
    else:
        upper = upper_posx
    for i in range(lower+1, upper):
        current_cutoffs = cutoffs[i]
        negx_cutoff = current_cutoffs[0]
        posx_cutoff = current_cutoffs[1]
        current_slice = slices[i]
        if upper_negx > upper_posx:
            if i < upper_posx and not posx_cutoff == None:
                current_slice = current_slice[current_slice[:,0] < posx_cutoff]
            if not negx_cutoff == None:
                current_slice = current_slice[current_slice[:,0] > negx_cutoff]
        else:
            if i < upper_negx and not negx_cutoff == None:
                current_slice = current_slice[current_slice[:,0] > negx_cutoff]
            if not posx_cutoff == None:
                current_slice = current_slice[current_slice[:,0] < posx_cutoff]
        new_slices.append(current_slice)
        f.value = i
    for i in range(upper, len(slices)):
        new_slices.append(slices[i])
        f.value = i
    return new_slices

def refine_slices(slices, max_neighbors, density, plot):
    print('|Refining Slices|\n')
    cutoffs = slice_cutoffs(slices, max_neighbors)
    
    lower = None
    middle = None
    upper_negx = None
    upper_posx = None
    count = 0
    print(cutoffs)
    for i in range(len(cutoffs)):
        current_cutoffs = cutoffs[i]
        negx_cutoff = current_cutoffs[0]
        posx_cutoff = current_cutoffs[1]
        if lower == None:
            if (negx_cutoff == None or posx_cutoff == None):
                lower = i
        elif middle == None:
            if not negx_cutoff == None and not posx_cutoff == None:
                if count == int(3*density)+1:
                    middle = i - count
                else:
                    count += 1
            else:
                count = 0
        else: 
            if negx_cutoff == None and upper_negx == None:
                upper_negx = i
            if posx_cutoff == None and upper_posx == None:
                upper_posx = i
                
    assert not lower == None, 'Lower Bound does not exist'
    assert not middle == None, 'Middle Bound does not exist'
    assert not upper_negx == None, 'Upper Negative X Bound does not exist'
    assert not upper_posx == None, 'Upper Positive X Bound does not exist'
    
    lower = np.asarray(cutoffs[:lower])
    negx = lower[:,0].astype(np.float64)
    posx = lower[:,1].astype(np.float64)
    m1, b1 = np.polyfit(range(len(negx)), negx, 1)
    m2, b2 = np.polyfit(range(len(posx)), posx, 1)
    lower = int((b2-b1)/(m1-m2))
    
    new_slices = cut_slices(slices, cutoffs, lower, upper_negx, upper_posx)
    new_points = np.array([0,0,0])
    for slice_points in new_slices:
        if len(slice_points) > 0:
            new_points = np.vstack((new_points,slice_points))
    new_points = new_points[1:]
    print('\n|Finished Refining Slices|\n')
    print(f'Number of Slices: {len(new_slices)}')
    print(f'Number of Points: {len(new_points)}')
    if plot:
        x1 = np.array([i for i in range(len(negx))])
        y1 = negx
        plt.plot(x1,y1, 'yo', x1, m1*x1+b1, '--k')
        x2 = np.array([i for i in range(len(posx))])
        y2 = posx
        plt.plot(x2,y2, 'yo', x2, m2*x2+b2, '--k')
        plt.title('X cutoffs over the Lower-body Slices')
        plt.show()
        
    return new_points, new_slices

In [20]:
def knn_down_sample(points, num_neighbors):
    tree = KDTreeFlann(create_pcd(points))
    new_points = np.array([0,0,0])
    for point in points:
        [_, neighbor_idx, _] = tree.search_knn_vector_3d(point, num_neighbors)
        mean_points = point
        for neighbor in neighbor_idx:
            mean_points = np.vstack((mean_points, points[neighbor]))
        new_points = np.vstack((new_points, mean_points.mean(axis=0)))    
    return new_points[1:]

def slice_tolerance(points, neighbors):
    distances = []
    point_idx = points[:,1].argsort()
    initial = point_idx[0]
    for idx in point_idx:
        current_point = points[idx]
        current_y = current_point[1]
        if idx == initial:
            points_above = points[points[:,1] > current_y]
        else:
            points_above = points_above[points_above[:,1] > current_y]
        if len(points_above) >= 2:
            tree = KDTreeFlann(create_pcd(points_above))
            [_, neighbor_idx, _] = tree.search_knn_vector_3d(current_point, neighbors)
            for idx in neighbor_idx:
                distances.append(points_above[idx][1] - current_y)      
    return max(distances)

def intersection(p1, p2, y):
    x1, y1, z1 = p1
    x2, y2, z2 = p2
    new_x = ((y - y1)*(x2 - x1)/(y2 - y1)) + x1
    new_z = ((y - y1)*(z2 - z1)/(y2 - y1)) + z1
    return np.array([new_x, y, new_z])

def slice_triangulation(pcd, slice_neighbors=2, density=2, down_sample_neighbors=6, voxel_size=10, refined=True, segmentation_neighbors=2, plot=True):
    points = np.asarray(pcd.points)
    
    tolerance = 0.5*slice_tolerance(points, slice_neighbors)
    increment = tolerance/density
    min_y = points[:,1].min()
    max_y = points[:,1].max()
    current_y = min_y

    num_sections = 1
    sections = []
    for i in range(num_sections):
        p1 = i*(100/num_sections)
        p2 = (i+1)*(100/num_sections)
        q1 = np.percentile(points[:,1],p1)
        q2 = np.percentile(points[:,1],p2)
        current_section = points[points[:,1] >= q1 - (tolerance+increment)]
        current_section = current_section[current_section[:,1] <= q2 + (tolerance+increment)]
        sections.append([q2,current_section])
    
    slices = []
    slice_points = np.array([0,0,0])
    print('|Triangulating Point Cloud|\n')
    print(f'Tolerance: {tolerance}')
    
    f = FloatProgress(min=min_y, max=max_y)
    display(f)
    
    for section in sections:
        current_max = section[0]
        current_section = section[1]
        while current_y <= current_max:
            current_section = current_section[current_section[:,1] >= current_y - tolerance]
            current_interval = current_section[current_section[:,1] <= current_y + tolerance]
            pts_above = current_interval[current_interval[:,1] > current_y]
            current_slice = np.array([0,0,0])
            if len(pts_above) > 0:
                pts_below = current_interval[current_interval[:,1] < current_y]
                pts_intersect = current_interval[current_interval[:,1] == current_y]
                tree = KDTreeFlann(create_pcd(pts_above))
                for pt_below in pts_below:
                    [_, neighbor_idx, _] = tree.search_knn_vector_3d(pt_below, slice_neighbors)
                    for idx in neighbor_idx:
                        current_slice = np.vstack((current_slice, intersection(pts_above[idx], pt_below, current_y)))
                for pt_intersect in pts_intersect:
                    current_slice = np.vstack((current_slice, pt_intersect))
            if len(current_slice.shape) == 2:
                current_slice = np.unique(current_slice[1:],axis=0)
                if len(current_slice) > 4:
                    current_slice = knn_down_sample(current_slice, down_sample_neighbors)
                    current_slice = np.asarray(voxel_down_sample(create_pcd(current_slice), voxel_size).points)
                    slices.append(current_slice)
                    slice_points = np.vstack((slice_points,current_slice))
            current_y += increment
            f.value = current_y
    f.value = max_y
    slice_points = slice_points[1:]
    print('|Finished Triangulations|\n')
    print(f'Number of Slices: {len(slices)}')
    print(f'Number of Points: {len(slice_points)}')
    if refined:
        return refine_slices(slices, segmentation_neighbors, density, plot)
    else:
        return slice_points, slices

In [21]:
folderdir = '/Users/nickf/Desktop/Custom-Fit/Data/Body-Datasets/Caesar-Data/CSRFM_PCD/'
filedirs = glob.glob(f'{folderdir}/*.ply')
file_num = 10
pcd = read_pcd(filedirs[file_num])
pcd = pcd_processing(pcd, 232)
slice_points, slices = slice_triangulation(pcd, slice_neighbors=6, density=4, down_sample_neighbors=6, voxel_size=15, segmentation_neighbors=1)





Point Cloud File successfully loaded
|Triangulating Point Cloud|

Tolerance: 22.35999560373716


FloatProgress(value=0.0, max=1796.1871076642772)

|Finished Triangulations|

Number of Slices: 321
Number of Points: 20921
|Refining Slices|

Finding Slice Cutoffs


FloatProgress(value=0.0, max=320.0)

[[None, None], [-86.8461200929232, 73.04462785162744], [-74.05782849299413, 62.231335095001704], [-72.63939770622514, 62.359025533442896], [-75.72419873236306, 62.43234317271717], [-73.4413532701553, 57.24896533935246], [-76.40524687512726, 58.751562995778954], [-77.37952483003487, 56.28344598303278], [-77.12035618813219, 53.573709614907955], [-79.13352738684267, 54.75911340555607], [-78.1992318827644, 53.96713707085986], [-77.99567566863979, 53.14683380121323], [-78.54992146487032, 54.16302641486421], [-77.55454678656346, 54.07645484316836], [-77.61919354388522, 53.27248524510716], [-75.86142358440395, 52.077626601664534], [-80.33364184535549, 52.317119386754214], [-80.82807055817781, 51.48062487228423], [-84.45922546624261, 56.323928793608424], [-84.27402849386998, 57.30179305819193], [-84.44930422440042, 56.51849951789211], [-87.6918116380841, 60.88140267611594], [-88.21576035355032, 62.789265164772495], [-89.87323517904058, 64.64473021008097], [-88.5547170404503, 64.83227908358927]

IndexError: too many indices for array

In [13]:
draw_geometries([create_pcd(slice_points)])

In [ ]:
def knn_down_sample(pcd, num_neighbors):
    tree = KDTreeFlann(pcd)
    points = np.array([0,0,0])
    pcd_points = np.asarray(pcd.points)
    for point in pcd_points:
        [_, neighbor_idx, _] = tree.search_knn_vector_3d(point, num_neighbors+1)
        mean_points = point
        for neighbor in neighbor_idx[1:]:
            mean_points = np.vstack((mean_points, pcd_points[neighbor]))
        points = np.vstack((points, mean_points.mean(axis=0)))    
    pcd.points = Vector3dVector(points[1:])
    return pcd

In [ ]:
correspondence_distance = 20
voxel_size = 15
num_neighbors = 10
source = create_pcd(slices[98])
target = create_pcd(slices[99])
source_pcd = knn_down_sample(source, num_neighbors)
target_pcd = knn_down_sample(target, num_neighbors)
source_pcd = voxel_down_sample(source_pcd, voxel_size)
target_pcd = voxel_down_sample(target_pcd, voxel_size)
draw_geometries([source_pcd, target_pcd])
correspondence = np.array(evaluate_registration(source_pcd,target_pcd,correspondence_distance).correspondence_set)
pcd_lineset = create_line_set_from_point_cloud_correspondences(source_pcd, target_pcd, correspondence)
correspondence = np.array(evaluate_registration(target_pcd, source_pcd,correspondence_distance).correspondence_set)
pcd_lineset_reverse = create_line_set_from_point_cloud_correspondences(target_pcd, source_pcd, correspondence)
draw_geometries([pcd_lineset,pcd_lineset_reverse])

In [ ]:
draw_geometries([create_pcd(new_points[1:])])

In [ ]:
draw_geometries([read_pcd('/Users/nickf/Desktop/Custom-Fit/Data/Subject Meshes/nf_1Spline.ply')])

In [ ]:
class ConcaveHull:
    
    def __init__(self):
        self.triangles = {}
        self.crs = {}
        
    
    def loadpoints(self, points):
        self.points = points
        
        
    def edge(self, key, triangle):
        '''Calculate the length of the triangle's outside edge
        and returns the [length, key]'''
        pos = triangle[1].index(-1)
        if pos==0:
            x1, y1 = self.points[triangle[0][0]]
            x2, y2 = self.points[triangle[0][1]]
        elif pos==1:
            x1, y1 = self.points[triangle[0][1]]
            x2, y2 = self.points[triangle[0][2]]
        elif pos==2:
            x1, y1 = self.points[triangle[0][0]]
            x2, y2 = self.points[triangle[0][2]]
        length = ((x1-x2)**2+(y1-y2)**2)**0.5
        rec = [length, key]
        return rec
        
    
    def triangulate(self):
        
        if len(self.points) < 2:
            raise Exception('CountError: You need at least 3 points to Triangulate')
        
        temp = list(zip(*self.points))
        x, y = list(temp[0]), list(temp[1])
        del(temp)
        
        triang = tri.Triangulation(x, y)
        
        self.triangles = {}
        
        for i, triangle in enumerate(triang.triangles):
            self.triangles[i] = [list(triangle), list(triang.neighbors[i])]
        

    def calculatehull(self, tol=20):
        
        self.tol = tol
        
        if len(self.triangles) == 0:
            self.triangulate()
        
        # All triangles with one boundary longer than the tolerance (self.tol)
        # is added to a sorted deletion list.
        # The list is kept sorted from according to the boundary edge's length
        # using bisect        
        deletion = []    
        self.boundary_vertices = set()
        for i, triangle in self.triangles.items():
            if -1 in triangle[1]:
                for pos, neigh in enumerate(triangle[1]):
                    if neigh == -1:
                        if pos == 0:
                            self.boundary_vertices.add(triangle[0][0])
                            self.boundary_vertices.add(triangle[0][1])
                        elif pos == 1:
                            self.boundary_vertices.add(triangle[0][1])
                            self.boundary_vertices.add(triangle[0][2])
                        elif pos == 2:
                            self.boundary_vertices.add(triangle[0][0])
                            self.boundary_vertices.add(triangle[0][2])
            if -1 in triangle[1] and triangle[1].count(-1) == 1:
                rec = self.edge(i, triangle)
                if rec[0] > self.tol and triangle[1].count(-1) == 1:
                    bisect.insort(deletion, rec)
                    
        while len(deletion) != 0:
            # The triangles with the longest boundary edges will be 
            # deleted first
            item = deletion.pop()
            ref = item[1]
            flag = 0
            
            # Triangle will not be deleted if it already has two boundary edges            
            if self.triangles[ref][1].count(-1) > 1:
                continue
                
            # Triangle will not be deleted if the inside node which is not
            # on this triangle's boundary is already on the boundary of 
            # another triangle
            adjust = {0: 2, 1: 0, 2: 1}            
            for i, neigh in enumerate(self.triangles[ref][1]):
                j = adjust[i]
                if neigh == -1 and self.triangles[ref][0][j] in self.boundary_vertices:
                    flag = 1
                    break
            if flag == 1:
                continue
           
            for i, neigh in enumerate(self.triangles[ref][1]):
                if neigh == -1:
                    continue
                pos = self.triangles[neigh][1].index(ref)
                self.triangles[neigh][1][pos] = -1
                rec = self.edge(neigh, self.triangles[neigh])
                if rec[0] > self.tol and self.triangles[rec[1]][1].count(-1) == 1:
                    bisect.insort(deletion, rec)
                    
            for pt in self.triangles[ref][0]:
                self.boundary_vertices.add(pt)
                                        
            del self.triangles[ref]
            
        self.polygon()
            
                    

    def polygon(self):
        
        edgelines = []
        boundary_edges = []
        boundary_points = []
        for i, triangle in self.triangles.items():
            if -1 in triangle[1]:
                for pos, value in enumerate(triangle[1]):
                    if value == -1:
                        if pos==0:
                            x1, y1 = self.points[triangle[0][0]]
                            x2, y2 = self.points[triangle[0][1]]
                        elif pos==1:
                            x1, y1 = self.points[triangle[0][1]]
                            x2, y2 = self.points[triangle[0][2]]
                        elif pos==2:
                            x1, y1 = self.points[triangle[0][0]]
                            x2, y2 = self.points[triangle[0][2]]
                        line = LineString([(x1, y1), (x2, y2)])
                        edgelines.append(line)
                        boundary_edges.append([len(boundary_points), len(boundary_points)+1])
                        boundary_points.append([x1, y1])
                        boundary_points.append([x2, y2])

        bound = linemerge(edgelines)
        self.boundary_edges = boundary_edges
        self.boundary_points = boundary_points
        self.boundary = Polygon(bound.coords)

def order_lineset(lineset,idx,used):
    if not idx in used:
        used.append(idx)
    for pt in lineset[idx]:
        if not pt in used:
            next_idx = pt
    used.append(next_idx)
    if len(used) == len(lineset):
        return used
    return order_lineset(lineset,next_idx,used)

In [ ]:
def control_points(s, max_neighbors):
    ch = ConcaveHull()
    ch.loadpoints(np.array([s[:,0],s[:,2]]).transpose())
    ch.calculatehull(tol=closest_distance(s, max_neighbors))
    pts = np.asarray(ch.boundary_points)

    pts_unique = np.unique(pts,axis=0)
    index = {}
    for idx in range(len(pts)):
        index[idx] = np.argwhere((pts_unique == pts[idx]).all(axis=1))[0][0]

    edges = []
    for edge in ch.boundary_edges:
        edges.append([index[edge[0]], index[edge[1]]])
    edges = np.asarray(edges)

    lineset = {}
    for i in range(len(edges)):
        current_edges = edges[(edges == i).any(axis=1)]
        lineset[i] = [current_edges[0, np.where(~(current_edges == i)[0])[0]][0],current_edges[1, np.where(~(current_edges == i)[1])[0]][0]]

    return pts_unique[order_lineset(lineset, list(lineset.keys())[0], list())]
    
def bspline(slices, degree=3, n=10000, neighbors=4):
    spline_slices = []
    spline_points = np.array([0,0,0])
    periodic=True
    f = FloatProgress(min=0, max=len(slices))
    display(f)
    for current_slice in slices:
        cp = control_points(current_slice, neighbors)
        count = cp.shape[0]
        kv = np.arange(-degree,count+degree+1)
        factor, fraction = divmod(count+degree+1, count)
        cp = np.roll(np.concatenate((cp,) * factor + (cp[:fraction],)),-1,axis=0)
        degree = np.clip(degree,1,degree)
        max_param = count - (degree * (1-periodic))
        spl = si.BSpline(kv, cp, degree)

        points = spl(np.linspace(0,max_param,n))
        current_y = np.array([current_slice[0,1] for _ in range(n)])
        points = np.array([points[:,0],current_y,points[:,1]]).transpose()
        spline_slices.append(points)
        spline_points = np.vstack((spline_points, points))
        
        f.value += 1
    return spline_points[1:], spline_slices

In [ ]:
spline_points, spline_slices = bspline(slices, degree=6, n=10000, neighbors=5)

In [ ]:
draw_geometries([create_pcd(spline_points)])

In [ ]:
filedir = '/Users/nickf/Desktop/Custom-Fit/Data/Subject Meshes/nf1_NewSpline.ply'
#write_pcd(create_pcd(all_points), filedir)
write_pcd(create_pcd(spline_points), filedir)